In [1]:
import pandas as pd

rows = [
["Bhai tum pagal ho","Mild Toxic"],
["This larka bohot achha hai","Safe"],
["Lan*t lagti ho","Abusive"],
["Tum really smart ho","Safe"],
["Chal bhaag yahan se","Offensive"]
]

df = pd.DataFrame(rows, columns=["comment","label"])
df.to_csv("starter_dataset.csv", index=False)
df.head()


,comment,label
0,Bhai tum pagal ho,Mild Toxic
1,This larka bohot achha hai,Safe
2,Lan*t lagti ho,Abusive
3,Tum really smart ho,Safe
4,Chal bhaag yahan se,Offensive


In [3]:
!pip install transformers datasets seqeval >/dev/null


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from datasets import Dataset


In [6]:
df = pd.read_csv("starter_dataset.csv")
df.head()


,comment,label
0,Bhai tum pagal ho,Mild Toxic
1,This larka bohot achha hai,Safe
2,Lan*t lagti ho,Abusive
3,Tum really smart ho,Safe
4,Chal bhaag yahan se,Offensive


In [11]:
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Safe tokenize function
def tokenize_fn(examples):
    # If batched=True, examples["comment"] is a list
    comments = [c if isinstance(c, str) else "" for c in examples["comment"]]
    return tokenizer(comments, truncation=True, padding='max_length', max_length=128)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [13]:
def tokenize_fn(examples):
    # examples["comment"] is a list of comments
    comments = [c if isinstance(c, str) else "" for c in examples["comment"]]
    return tokenizer(comments, truncation=True, padding='max_length', max_length=128)



In [15]:
from datasets import Dataset

# Convert pandas to HuggingFace dataset
train_ds = Dataset.from_pandas(train[['comment','label_id']])
test_ds  = Dataset.from_pandas(test[['comment','label_id']])

# Map using safe tokenize_fn
train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds  = test_ds.map(tokenize_fn, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 4  # Safe, Mild Toxic, Offensive, Abusive
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
!pip install --upgrade transformers datasets >/dev/null


In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer


In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_dir="./logs",
    logging_steps=5,
    save_strategy="no",
    seed=42,
    learning_rate=2e-5
)


In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    # eval_dataset=test_ds  # optional, demo ke liye skip
)


In [26]:
train_ds = train_ds.rename_column("label_id","labels")
test_ds  = test_ds.rename_column("label_id","labels")

train_ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])
test_ds.set_format(type='torch', columns=['input_ids','attention_mask','labels'])


In [27]:
print(train_ds[0])


{'labels': tensor(2), 'input_ids': tensor([  101, 53453, 10161,   170, 55200, 10240, 10549, 11781, 10126,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [29]:
def tokenize_fn(examples):
    comments = [str(c) for c in examples["comment"]]
    labels = examples["labels"]
    tokenized = tokenizer(comments, truncation=True, padding='max_length', max_length=128)
    tokenized["labels"] = labels
    return tokenized

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds  = test_ds.map(tokenize_fn, batched=True)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds
)

trainer.train()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: waffashahzad333 (waffashahzad333-government-college-woman-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.378009557723999, metrics={'train_runtime': 38.6063, 'train_samples_per_second': 0.104, 'train_steps_per_second': 0.026, 'total_flos': 263115780096.0, 'train_loss': 1.378009557723999, 'epoch': 1.0})

In [31]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=1, training_loss=1.3379175662994385, metrics={'train_runtime': 14.507, 'train_samples_per_second': 0.276, 'train_steps_per_second': 0.069, 'total_flos': 263115780096.0, 'train_loss': 1.3379175662994385, 'epoch': 1.0})

In [32]:
model.save_pretrained("Roman_Urdu_Toxicity_Model")
tokenizer.save_pretrained("Roman_Urdu_Toxicity_Model")


('Roman_Urdu_Toxicity_Model/tokenizer_config.json',
 'Roman_Urdu_Toxicity_Model/special_tokens_map.json',
 'Roman_Urdu_Toxicity_Model/vocab.txt',
 'Roman_Urdu_Toxicity_Model/added_tokens.json',
 'Roman_Urdu_Toxicity_Model/tokenizer.json')

In [33]:
from transformers import pipeline

pipe = pipeline("text-classification", model="Roman_Urdu_Toxicity_Model", tokenizer="Roman_Urdu_Toxicity_Model", return_all_scores=True)

examples = [
    "Bhai tu pagal ho",
    "This larka bohot achha hai",
    "Lan*t lagti ho",
    "I will kill you",
    "Tum bohot achay ho"
]

for ex in examples:
    res = pipe(ex)[0]
    sorted_res = sorted(res, key=lambda x: x['score'], reverse=True)
    print("Input:", ex)
    print(sorted_res[:3], "\n")


Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Input: Bhai tu pagal ho
[{'label': 'LABEL_1', 'score': 0.29635119438171387}, {'label': 'LABEL_2', 'score': 0.2922188639640808}, {'label': 'LABEL_0', 'score': 0.25031381845474243}] 

Input: This larka bohot achha hai
[{'label': 'LABEL_2', 'score': 0.3003489077091217}, {'label': 'LABEL_1', 'score': 0.30023306608200073}, {'label': 'LABEL_0', 'score': 0.23373594880104065}] 

Input: Lan*t lagti ho
[{'label': 'LABEL_2', 'score': 0.2801261842250824}, {'label': 'LABEL_1', 'score': 0.25471585988998413}, {'label': 'LABEL_0', 'score': 0.23742729425430298}] 

Input: I will kill you
[{'label': 'LABEL_2', 'score': 0.3002002537250519}, {'label': 'LABEL_1', 'score': 0.2601209282875061}, {'label': 'LABEL_0', 'score': 0.25263503193855286}] 

Input: Tum bohot achay ho
[{'label': 'LABEL_2', 'score': 0.2909373641014099}, {'label': 'LABEL_1', 'score': 0.2825074791908264}, {'label': 'LABEL_0', 'score': 0.2595527768135071}] 



In [34]:
model.save_pretrained("Roman_Urdu_Toxicity_Model")
tokenizer.save_pretrained("Roman_Urdu_Toxicity_Model")


('Roman_Urdu_Toxicity_Model/tokenizer_config.json',
 'Roman_Urdu_Toxicity_Model/special_tokens_map.json',
 'Roman_Urdu_Toxicity_Model/vocab.txt',
 'Roman_Urdu_Toxicity_Model/added_tokens.json',
 'Roman_Urdu_Toxicity_Model/tokenizer.json')